# Visualize data on an interactive map

This notebook uses `lonboard` for interactive visualisation of data.

In [1]:
import geopandas as gpd
import numpy as np
import lonboard
from core.utils import used_keys
from lonboard.colormap import apply_continuous_cmap
import matplotlib as mpl
from mapclassify import classify
from sidecar import Sidecar


Define data path

In [2]:
chars_dir = "/data/uscuni-ulce/processed_data/chars/"

Define region

In [3]:
region = 69333

## Buildings
Load building data and ensure the geometries are all valid Polygons.

In [53]:
buildings = gpd.read_parquet(f"{chars_dir}buildings_chars_{region}.parquet").to_crs(4326).reset_index()

buildings.geometry = buildings.make_valid()

buildings = buildings[buildings.geom_type.str.contains("Polygon")]

Create a lonboard layer

In [54]:
%%time
layer = lonboard.SolidPolygonLayer.from_geopandas(buildings, opacity=.3)

CPU times: user 1.69 s, sys: 193 ms, total: 1.88 s
Wall time: 1.87 s


Create a Sidecar view (assumes JupyterLab) for more comfortable experience.

In [55]:
sc = Sidecar(title='buildings')

Create a Map object

In [56]:
m = lonboard.Map(layer)

Display map within the sidecar plugin

In [57]:
with sc:
    display(m)

List avaialable columns

In [58]:
buildings.columns

Index(['level_0', 'index', 'currentUse', 'buildingNature', 'heightAboveGround',
       'iid', 'geometry', 'ssbCCo', 'ssbCor', 'ssbSqu', 'ssbCCM', 'ssbCCD',
       'sdbAre', 'sdbPer', 'sdbCoA', 'ssbERI', 'ssbElo', 'stbOri', 'mtbSWR',
       'libNCo', 'ldbPWL', 'ltcBuA', 'mtbAli', 'mtbNDi', 'ltbIBD', 'stbCeA',
       'nID', 'stbSAl', 'nodeID'],
      dtype='object')

Specify a column and pass its values into a choropleth representation within the map. 

In [59]:
# buildings.explore()

In [116]:
column = 'stbSAl'

classifier = classify(buildings[column], 'quantiles', k=20)
normalizer = mpl.colors.Normalize(0, classifier.bins.shape[0])
vals = normalizer(classifier.yb)
layer.get_fill_color = apply_continuous_cmap(vals, mpl.colormaps['viridis'])

In [117]:
used_keys[column]

'street alignment of building'

In [118]:
buildings[column].describe().iloc[1:]

mean     8.943344
std      9.662884
min      0.000019
25%      1.251941
50%      5.241196
75%     13.794330
max     44.965289
Name: stbSAl, dtype: float64

## Tessellation

Load tessellation data and ensure the geometries are all valid Polygons.

In [11]:
tess = gpd.read_parquet(f"{chars_dir}tessellations_chars_{region}.parquet").to_crs(4326)

tess.geometry = tess.make_valid()

tess = tess[tess.geom_type.str.contains("Polygon")]

Create a lonboard layer

In [12]:
%%time
layer = lonboard.SolidPolygonLayer.from_geopandas(tess, opacity=.2)

CPU times: user 5.26 s, sys: 307 ms, total: 5.57 s
Wall time: 5.57 s


Create a Sidecar view (assumes JupyterLab) for more comfortable experience.

In [13]:
sc = Sidecar(title='tess')

Create a Map object

In [14]:
m = lonboard.Map(layer)

Display map within the sidecar plugin

In [15]:
with sc:
    display(m)

List avaialable columns

In [16]:
tess.columns

Index(['enclosure_index', 'geometry', 'stcOri', 'sdcLAL', 'sdcAre', 'sscCCo',
       'sscERI', 'mtcWNe', 'mdcAre', 'ltcWRB', 'sicCAR', 'stcSAl', 'nID',
       'nodeID'],
      dtype='object')

In [51]:
column = 'stcSAl'
used_keys[column]

'street alignment of ETC'

Specify a column and pass its values into a choropleth representation within the map. 

In [52]:

classifier = classify(tess[column], 'quantiles', k=20)
normalizer = mpl.colors.Normalize(0, classifier.bins.shape[0])
vals = normalizer(classifier.yb)
layer.get_fill_color = apply_continuous_cmap(vals, mpl.colormaps['viridis'])

## Enclosures

Load data and ensure the geometries are all valid Polygons.

In [20]:
enc = gpd.read_parquet(f"{chars_dir}enclosures_chars_{region}.parquet").to_crs(4326)

enc.geometry = enc.make_valid()

enc = enc[enc.geom_type.str.contains("Polygon")]

Create a lonboard layer

In [30]:
%%time
layer = lonboard.SolidPolygonLayer.from_geopandas(enc, opacity=.3)

CPU times: user 380 ms, sys: 48 ms, total: 428 ms
Wall time: 427 ms


Create a Sidecar view (assumes JupyterLab) for more comfortable experience.

In [31]:
sc = Sidecar(title='enclosures')

Create a Map object

In [32]:
m = lonboard.Map(layer)

Display map within the sidecar plugin

In [33]:
with sc:
    display(m)

List avaialable columns

In [34]:
enc.columns

Index(['eID', 'geometry', 'ldkAre', 'ldkPer', 'lskCCo', 'lskERI', 'lskCWA',
       'ltkOri', 'ltkWNB', 'likWCe', 'likWBB'],
      dtype='object')

In [41]:
column = 'ldkPer'
used_keys[column]

'perimeter of enclosure'

Specify a column and pass its values into a choropleth representation within the map. 

In [42]:

classifier = classify(enc[column], 'quantiles', k=20)
normalizer = mpl.colors.Normalize(0, classifier.bins.shape[0])
vals = normalizer(classifier.yb)
layer.get_fill_color = apply_continuous_cmap(vals, mpl.colormaps['viridis'])

## Streets

Load data and ensure the geometries are all valid Polygons.

In [119]:
streets = gpd.read_parquet(f"{chars_dir}streets_chars_{region}.parquet")

streets.geometry = streets.make_valid()

Create a lonboard layer

In [120]:
%%time
layer = lonboard.PathLayer.from_geopandas(streets.to_crs(4326), width_min_pixels=1)

CPU times: user 254 ms, sys: 33 ms, total: 287 ms
Wall time: 285 ms


Create a Sidecar view (assumes JupyterLab) for more comfortable experience.

In [121]:
sc = Sidecar(title='streets')

Create a Map object

In [122]:
m = lonboard.Map(layer)

Display map within the sidecar plugin

In [123]:
with sc:
    display(m)

List avaialable columns

In [ ]:
assert np.allclose(streets['sdsLen'] , streets.geometry.length)

if 'mm_len' in streets.columns:
    assert np.allclose(streets['mm_len'] , streets.geometry.length)

In [124]:
streets.columns

Index(['geometry', '_status', 'mm_len', 'cdsbool', 'node_start', 'node_end',
       'sdsLen', 'sssLin', 'ldsMSL', 'sdsAre', 'ldsRea', 'ldsAre', 'sisBpM',
       'sdsSPW', 'sdsSPO', 'sdsSWD', 'nID'],
      dtype='object')

In [125]:
streets[['sdsLen', 'sssLin', 'ldsMSL', 'sdsAre', 'ldsRea', 'ldsAre',
       'sisBpM', 'sdsSPW', 'sdsSPO', 'sdsSWD']].describe()

,sdsLen,sssLin,ldsMSL,sdsAre,ldsRea,ldsAre,sisBpM,sdsSPW,sdsSPO,sdsSWD
count,79382.000000,79382.000000,79382.000000,5.434800e+04,79382.000000,6.509000e+04,54353.000000,79382.000000,79382.000000,58304.000000
mean,190.217746,0.959236,204.381858,2.216389e+04,122.845935,3.741573e+05,0.059933,33.719737,0.744487,3.816754
std,362.683313,0.105247,170.864389,4.742172e+04,99.856645,3.653913e+05,0.079351,12.298330,0.224721,1.965380
min,2.000000,0.000000,3.741196,1.738884e-04,0.000000,4.620473e+01,0.000181,0.000000,0.000000,0.000000
25%,59.682238,0.975660,105.168879,3.535433e+03,46.000000,1.571747e+05,0.027543,23.104647,0.580000,2.394241
50%,99.256838,0.998659,137.507663,8.279731e+03,115.000000,2.505989e+05,0.049314,31.799232,0.750000,3.939210
75%,174.469390,1.000000,228.649039,2.051356e+04,181.000000,4.457882e+05,0.076658,50.000000,1.000000,5.253920
max,11193.902019,1.000000,3118.856646,1.373843e+06,1493.000000,4.690208e+06,7.008423,50.000000,1.000000,11.162694


Specify a column and pass its values into a choropleth representation within the map. 

In [148]:
column = 'sdsSPW'
used_keys[column]

'width of street profile'

In [149]:
streets[column] = streets[column].fillna(0)

In [150]:
classifier = classify(streets[column].astype(int), 'equalinterval', k=20)
normalizer = mpl.colors.Normalize(0, classifier.bins.shape[0])
vals = normalizer(classifier.yb)
layer.get_color = apply_continuous_cmap(vals, mpl.colormaps['viridis'])

## Nodes

Load data and ensure the geometries are all valid Polygons.

In [151]:
nodes = gpd.read_parquet(f"{chars_dir}nodes_chars_{region}.parquet").to_crs(4326)

Create a lonboard layer

In [152]:
%%time
layer = lonboard.ScatterplotLayer.from_geopandas(nodes, radius_min_pixels=2)

CPU times: user 69.3 ms, sys: 4.83 ms, total: 74.2 ms
Wall time: 73.6 ms


Create a Sidecar view (assumes JupyterLab) for more comfortable experience.

In [153]:
sc = Sidecar(title='nodes')

Create a Map object

In [154]:
m = lonboard.Map(layer, basemap_style=lonboard.basemap.CartoBasemap.Positron)

Display map within the sidecar plugin

In [155]:
with sc:
    display(m)

List avaialable columns

In [156]:
nodes.columns

Index(['x', 'y', 'mtdDeg', 'lcdMes', 'linP3W', 'linP4W', 'linPDE', 'lcnClo',
       'lddNDe', 'linWID', 'ldsCDL', 'xcnSCl', 'mtdMDi', 'nodeID', 'geometry',
       'sddAre', 'midRea', 'midAre'],
      dtype='object')

Specify a column and pass its values into a choropleth representation within the map. 

In [161]:
nodes[['mtdDeg', 'lcdMes', 'linP3W', 'linP4W', 'linPDE', 'lcnClo',
       'lddNDe', 'linWID', 'ldsCDL', 'xcnSCl', 'mtdMDi', 'nodeID', 'geometry',
       'sddAre', 'midRea', 'midAre']].describe()

,mtdDeg,lcdMes,linP3W,linP4W,linPDE,lcnClo,lddNDe,linWID,ldsCDL,xcnSCl,mtdMDi,nodeID,sddAre,midRea,midAre
count,61838.000000,61838.000000,61838.000000,61838.000000,61838.000000,6.183800e+04,61838.000000,61838.000000,61838.000000,61838.000000,61838.000000,61838.00000,4.654500e+04,61838.000000,4.908000e+04
mean,2.567418,0.105057,0.683189,0.130161,0.183928,1.004460e-06,0.005973,0.009984,319.454293,0.027892,183.852532,30918.50000,2.587954e+04,20.845176,6.426385e+04
std,1.005752,0.060368,0.116292,0.098865,0.115463,6.709700e-07,0.007673,0.005341,612.723878,0.053212,280.458199,17851.23731,4.584542e+04,23.542853,8.924184e+04
min,1.000000,-1.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000396,0.000000,0.000000,0.000000,2.311779,0.00000,1.159084e+00,0.000000,1.221930e+01
25%,1.000000,0.061856,0.625000,0.058824,0.111111,4.621507e-07,0.003492,0.005882,73.066534,0.000000,74.482961,15459.25000,5.936987e+03,3.000000,1.793014e+04
50%,3.000000,0.098592,0.695652,0.111111,0.169492,8.682536e-07,0.005624,0.009957,181.984598,0.000000,109.536100,30918.50000,1.170070e+04,16.000000,3.585703e+04
75%,3.000000,0.142857,0.757576,0.181818,0.238095,1.420323e-06,0.007255,0.013344,364.331816,0.047619,170.624712,46377.75000,2.549037e+04,31.000000,7.125945e+04
max,6.000000,0.800000,1.000000,0.777778,1.000000,7.283451e-06,0.534588,0.119944,22877.000260,1.000000,11193.902019,61837.00000,1.170909e+06,583.000000,1.806340e+06


In [185]:
column = 'mtdMDi'
used_keys[column]

'mean distance to neighbouring nodes of street network'

In [186]:

classifier = classify(nodes[column], 'quantiles', k=20)
normalizer = mpl.colors.Normalize(0, classifier.bins.shape[0])
vals = normalizer(classifier.yb)
layer.get_fill_color = apply_continuous_cmap(vals, mpl.colormaps['viridis'])